In order to use dataframes and the SQL engine in Apache Spark (SparkSQL), we need a Spark Session.


In [31]:
import pyspark

In [32]:
from pyspark.sql import SparkSession

In [33]:
spark = SparkSession.builder.appName('Practice').getOrCreate()

In [34]:
spark

## Spark Dataframes

### Read dataset as Spark dataframe from file

In [35]:
# Read Dataset as Spark dataframe from json file
df = spark.read.json('people.json')

In [36]:
df.show()

+---+-------+
|age|   name|
+---+-------+
| 25|Michael|
| 24|   Andy|
| 19| Justin|
| 26| George|
| 30|   Jeff|
+---+-------+



In [37]:
# Read Dataset as Spark dataframe from csv file
df = spark.read.csv('people.csv')

In [38]:
df.show()

+-------+----+
|    _c0| _c1|
+-------+----+
|   name| age|
|Michael|  25|
|   Andy|  24|
| Justin|  19|
| George|  26|
|   Jeff|  30|
+-------+----+



In [21]:
df.head(2)

[Row(age=25, name='Michael'), Row(age=24, name='Andy')]

In [22]:
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [23]:
df.select("name").show()

+-------+
|   name|
+-------+
|Michael|
|   Andy|
| Justin|
| George|
|   Jeff|
+-------+



In [24]:
df.select(df["name"]).show()

+-------+
|   name|
+-------+
|Michael|
|   Andy|
| Justin|
| George|
|   Jeff|
+-------+



In [27]:
# filtering
df.filter(df["age"] > 21).show()

+---+-------+
|age|   name|
+---+-------+
| 25|Michael|
| 24|   Andy|
| 26| George|
| 30|   Jeff|
+---+-------+



In [28]:
# aggregation of data
df.groupBy("age").count().show()

+---+-----+
|age|count|
+---+-----+
| 26|    1|
| 19|    1|
| 25|    1|
| 30|    1|
| 24|    1|
+---+-----+



In [29]:
spark.stop()